In [1]:
F = GF(11)
F_nonzero = [i for i in F if i != 0]

In [2]:
P.<x> = F[]

In [3]:
n = 10
t = 5

In [4]:
def share(n, t, secret):
    g = choice(F_nonzero)*x^(t-1) + sum([x^i*F.random_element() for i in range(1, t-1)]) + secret
    print(g)
    points = [g(i) for i in range (1, n+1)]
    return points

In [5]:
shares = share(n, t, 7)
shares

4*x^4 + 3*x^3 + 6*x^2 + 10*x + 7


[8, 7, 1, 6, 2, 10, 4, 10, 7, 4]

In [6]:
def reconstruct(parties, shares):
    g = P.lagrange_polynomial([(parties[i], shares[i]) for i in range (0, len(parties))])
    print(g)
    return g(0)

In [7]:
reconstruct([*range(1,6)], shares[:5])

4*x^4 + 3*x^3 + 6*x^2 + 10*x + 7


7

In [8]:
secrets = [2, 2, 3, 3, 0, 0, 0, 0, 0, 0]

In [9]:
Shares = [share(n, t, secrets[i]) for i in range(n)]
Shares

8*x^4 + 5*x^3 + 2*x^2 + 2*x + 2
10*x^4 + x^2 + 9*x + 2
6*x^4 + 5*x^3 + 8*x^2 + 4*x + 3
10*x^4 + 7*x^3 + 9*x^2 + 6*x + 3
10*x^4 + 4*x^3 + 9*x^2 + 3*x
9*x^4 + 4*x^3 + 2*x^2 + 8*x
2*x^4 + 9*x^3 + 7*x^2 + 9*x
10*x^4 + 3*x^3 + 9*x^2 + 2*x
8*x^4 + 3*x^3 + 3*x^2 + 6*x
7*x^4 + 8*x^3 + 4*x^2 + x


[[8, 6, 6, 1, 0, 6, 5, 10, 6, 5],
 [0, 8, 1, 7, 8, 6, 1, 2, 5, 4],
 [4, 3, 4, 1, 0, 8, 0, 7, 6, 8],
 [2, 3, 1, 0, 2, 7, 2, 5, 10, 9],
 [4, 3, 7, 2, 5, 9, 5, 4, 4, 1],
 [1, 2, 10, 6, 0, 9, 2, 10, 5, 10],
 [5, 7, 0, 4, 10, 2, 1, 10, 3, 2],
 [2, 4, 10, 0, 7, 7, 7, 1, 3, 3],
 [9, 0, 4, 2, 2, 6, 10, 4, 5, 2],
 [9, 7, 8, 7, 2, 5, 9, 10, 7, 2]]

In [10]:
def beta(i, y_i):
    return prod([-F(j) / F(i - j) if j != i else 1 for j in range(1, n+1)]) * F(y_i)

In [11]:
x1x2 = vector(Shares[0]).pairwise_product(vector(Shares[1]))
x1x2

(0, 4, 6, 7, 0, 3, 5, 9, 8, 9)

In [12]:
#x1x2_beta = [beta(i, x1x2[i - 1]) for i in range (1, 2*t)]
x1x2_beta = [beta(i, x1x2[i - 1]) for i in range(1, n+1)]
x1x2_beta

[0, 7, 5, 4, 0, 8, 6, 2, 3, 2]

In [13]:
x1x2_shares = matrix([share(n, t, x1x2_beta[i-1]) for i in range (1, n+1)])
x1x2_shares

5*x^4 + 4*x^3 + x^2 + 10*x
5*x^4 + 10*x^3 + 9*x^2 + 5*x + 7
5*x^4 + 5*x^3 + 2*x^2 + 5*x + 5
8*x^4 + 9*x^3 + 6*x^2 + 10*x + 4
3*x^4 + 2*x^3 + 10*x^2 + 8*x
5*x^4 + 3*x^3 + 5*x^2 + x + 8
8*x^4 + 8*x^3 + 4*x^2 + x + 6
8*x^4 + 5*x^3 + 10*x^2 + 9*x + 2
7*x^4 + 3*x^3 + 10*x^2 + 9*x + 3
7*x^4 + 4*x^3 + 8*x^2 + 7*x + 2


[ 9  4  2  8  4  4 10  1 10  3]
[ 3  4  8  1  1  3  1 10  0  6]
[ 0  0  6  7  2  0  9  3 10  2]
[ 4  6  0  3  4  8  3  4  9 10]
[ 1 10  4 10  6  9  9  2  1  3]
[ 0  2  3  2  8  7  6  0  5  3]
[ 5  7  7  5  6  9  7  9  7  9]
[ 1  8  0  3  4  6  6  6  2  6]
[10 10  9  5 10  6  0  2  3  8]
[ 6  5  0  6  8  5 10  6  1  6]

In [14]:
x1x2_final = [
    sum(
        [x1x2_shares[i, j] for i in range (0, n)]
    ) for j in range (0, n)
]
x1x2_final

[6, 1, 6, 6, 9, 2, 6, 10, 4, 1]

In [15]:
reconstruct([*range(1,6)], x1x2_final[:5])

6*x^4 + 9*x^3 + 10*x^2 + 10*x + 4


4

In [16]:
y_i = vector(x1x2_final).pairwise_product(vector(Shares[2]))
y_i

(2, 3, 2, 6, 0, 5, 0, 4, 2, 8)

In [17]:
beta_x1x2x3 = [beta(i, y_i[i - 1]) for i in range(1, n+1)]
beta_x1x2x3

[9, 8, 9, 5, 0, 6, 0, 7, 9, 3]

In [18]:
share_x1x2x3 = matrix([share(n, t, beta_x1x2x3[i-1]) for i in range (1, n+1)])
share_x1x2x3

x^4 + 3*x^3 + 10*x^2 + x + 9
6*x^4 + 4*x^3 + x^2 + 6*x + 8
7*x^4 + 8*x^3 + 8*x^2 + 9*x + 9
4*x^4 + 7*x^3 + 10*x^2 + 5
2*x^4 + x^2 + x
9*x^4 + 2*x^3 + 4*x^2 + 9*x + 6
9*x^4 + 10*x^3 + 4*x^2 + 3*x
3*x^4 + 9*x^3 + 5*x^2 + 3*x + 7
7*x^4 + 4*x^3 + 10*x^2 + 5*x + 9
4*x^4 + x^3 + 7*x^2 + 10*x + 3


[ 2  3  0  5 10  9  9  8  6  5]
[ 3  9  2  3  1  8  4  3  9  5]
[ 8  4  0  2  8  8  6  9  5  7]
[ 4  0  3  9  0 10  4 10  9  1]
[ 4  5  9  4  4  5  7  3  1  2]
[ 8  2  5  8  9  2 10  8  0  8]
[ 4  4 10  6  5  5  0  2  8  0]
[ 5 10  8  2  1  9  3 10  8  3]
[ 2  5  8  4  0  6  2  4  9  6]
[ 3  2  7  0  4  6  1  3  1  3]

In [19]:
x1x2x3_shares = [
    sum(
        [share_x1x2x3[i, j] for i in range (0, n)]
    ) for j in range (0, n)
]
x1x2x3_shares

[10, 0, 8, 10, 9, 2, 2, 5, 1, 7]

In [20]:
reconstruct([*range(1,6)], x1x2x3_shares[:5])

8*x^4 + 4*x^3 + 5*x^2 + 3*x + 1


1

In [21]:
res_shares = vector(x1x2x3_shares) + vector(Shares[3])
res_shares

(1, 3, 9, 10, 0, 9, 4, 10, 0, 5)

In [22]:
reconstruct([*range(1,6)], res_shares[:5])

7*x^4 + 3*x^2 + 9*x + 4


4